In [2]:
!pip install -q fsspec==2025.3.0 gcsfs==2025.3.0 requests==2.32.3
!pip install --upgrade transformers peft bitsandbytes accelerate datasets trl
#accelerate==0.21.0 peft==0.4.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.43.0 trl==0.4.7

In [3]:
import transformers
import textwrap
import os
import sys
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    #prepare_model_for_int8_training,
)

import torch
from datasets import load_dataset
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams

%matplotlib inline
sns.set(rc={'figure.figsize':(10, 7)})
sns.set(rc={'figure.dpi':100})
sns.set(style='white', palette='muted', font_scale=1.2)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [4]:
!pip install -q huggingface_hub

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `llama 3.2 for email` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llama

In [6]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should be >= 1
print(torch.cuda.current_device())  # Should be 0 or another valid index
print(torch.cuda.get_device_name(0))  # Should show your GPU name


True
1
0
Tesla T4


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    quantization_config=bnb_config,
    device_map="balanced",
    torch_dtype=torch.float16
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
print(next(model.parameters()).device)

cuda:0


In [10]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fariaaupee","key":"57c8aefa5e7ccc52b64b3aa5b27d5f1c"}'}

In [11]:
import os
import zipfile

# Make a Kaggle directory and move the json file
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [12]:
!kaggle datasets download -d thedevastator/uncovering-enron-employees-secrets-exploring-the

Dataset URL: https://www.kaggle.com/datasets/thedevastator/uncovering-enron-employees-secrets-exploring-the
License(s): CC0-1.0
  0% 0.00/4.76M [00:00<?, ?B/s]
100% 4.76M/4.76M [00:00<00:00, 796MB/s]


In [13]:
!unzip -q '*.zip'

In [14]:
import pandas as pd
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("validation.csv")

In [15]:
df

,email_body,subject_line
0,"Greg/Phillip, Attached is the Grande Communic...",Service Agreement
1,Phillip & Keith Attached is the first draw re...,Bishops Corner
2,Your Internet Banking accounts are now setup a...,Internet Banking
3,To our IBS Customers that are still hanging in...,Internet Banking
4,Phillip Good Morning!\nI hope you had a wonder...,SMEs for expert stories
...,...,...
14431,This email is acknowledgement from the Power P...,Power Pool
14432,This email is acknowledgement from the Power P...,Power Pool of Alberta
14433,"John, Further to the voice message that I lef...",Enron Security
14434,Make sure that all curves are downloaded by th...,Simulation Curves


In [16]:
df = df[["email_body", "subject_line"]].dropna()
from datasets import Dataset

# Optional: use only first 1000 rows
df = df.head(10000)

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['email_body', 'subject_line'],
    num_rows: 10000
})

In [17]:
df_val = df_val.head(1000)
df_val = df_val[["email_body", "subject_line"]].dropna()
val_dataset = Dataset.from_pandas(df_val)
val_dataset

Dataset({
    features: ['email_body', 'subject_line'],
    num_rows: 1000
})

In [18]:
def format_for_sft(example):
    return {
        "text": f"### Instruction:\nGenerate a subject line for the following email body:\n\n### Email Body:\n{example['email_body']}\n\n### Subject Line:\n{example['subject_line']}"
    }

dataset = dataset.map(format_for_sft)
val_dataset = val_dataset.map(format_for_sft)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=1024)

tokenized_ds = dataset.map(tokenize)
tokenized_val_ds = val_dataset.map(tokenize)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [29]:
from peft import LoraConfig, get_peft_model
import torch

# Apply LoRA config
# model = prepare_model_for_int8_training(model)
peft_config = LoraConfig(
    r=4,
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,146,880 || all params: 3,213,896,704 || trainable%: 0.0357


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [30]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [31]:
from trl import SFTTrainer
from transformers import Trainer
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    logging_steps=50,
    learning_rate=2e-4,
    fp16=True,
    disable_tqdm=False,
    output_dir="./llama3-email-summary",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    num_train_epochs=3,
    save_strategy="steps",
    #eval_strategy="steps",
    save_total_limit=1,
    #load_best_model_at_end=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=val_dataset,
    #dataset_text_field="text",
    args=training_args,
    peft_config=peft_config,
    #max_seq_length=512,
    data_collator=data_collator,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    # packing=True,
    # tokenizer=tokenizer,
)

Converting train dataset to ChatML:   0%|          | 0/10000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/1000 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
print(transformers.__version__)

4.51.3


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aupeef (aupeef-khulna-university-of-engineering-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


Step,Training Loss
50,3.001700
100,2.634500
150,2.674300
200,2.601200
250,2.633700
300,2.615900
350,2.586500
400,2.584200
450,2.493400
500,2.571500


TrainOutput(global_step=3750, training_loss=2.4435106323242186, metrics={'train_runtime': 8532.967, 'train_samples_per_second': 3.516, 'train_steps_per_second': 0.439, 'total_flos': 1.2934523406097613e+17, 'train_loss': 2.4435106323242186})

In [20]:
print(len(df_test))
test_df = df_test.head(1000)
test_df = test_df[["email_body", "subject_line"]].dropna()
test_dataset = Dataset.from_pandas(test_df)
test_dataset

1906


Dataset({
    features: ['email_body', 'subject_line'],
    num_rows: 1000
})

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linea

In [32]:
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=1b4a3ffe4806ccb34f4b3fad36e572236120b777271dce307de9199cac491d14
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [33]:
# import evaluate

# rouge = evaluate.load("rouge")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     return {k: round(v.mid.fmeasure * 100, 2) for k, v in result.items()}

In [35]:
# trainer.compute_metrics = compute_metrics
# eval_results = trainer.evaluate()
# print(eval_results)

In [ ]:
import torch
torch.cuda.empty_cache()

# **Saving the fine tuned model and pushing into hf**

In [ ]:
from peft import PeftModel

# Save only the adapter (not full base model)
peft_model_path = "/content/llama3-email-summary"
model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('/content/llama3-email-summary/tokenizer_config.json',
 '/content/llama3-email-summary/special_tokens_map.json',
 '/content/llama3-email-summary/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("Llama3-email-subject")
tokenizer.push_to_hub("Llama3-email-subject")

adapter_model.safetensors:   0%|          | 0.00/4.60M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/faria-aupee/Llama3-email-subject/commit/5867d387889e1239d0aad27a2064e1e43ddf875f', commit_message='Upload tokenizer', commit_description='', oid='5867d387889e1239d0aad27a2064e1e43ddf875f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/faria-aupee/Llama3-email-subject', endpoint='https://huggingface.co', repo_type='model', repo_id='faria-aupee/Llama3-email-subject'), pr_revision=None, pr_num=None)

# **Loading the fine tuned peft model**

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

ft_tokenizer = AutoTokenizer.from_pretrained("faria-aupee/Llama3-email-subject")

ft_model = PeftModel.from_pretrained(model, "faria-aupee/Llama3-email-subject") #model = base model before fine tuning

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/4.60M [00:00<?, ?B/s]

# **Inference from fine tuned model alone**

In [40]:
from tqdm import tqdm
def format_prompt(example):
    return {
        "prompt": f"### Instruction:\nGenerate a complete and non-repetitive subject line for the following email body within 20 words.\
         Keep the subject concise and relevant.\
         Exclude any details, sender/recipient information and also exclude the full email body content from the subject line,\
         Generate a brief subject line only.\
         \n\n### Email Body:\n{example['email_body']}\n\n### Subject Line:"
    }

test_dataset = test_dataset.map(format_prompt)
preds = []
refs = []

# Inference loop
for i, example in enumerate(tqdm(test_dataset)):  # change range using .select(range(k))
    prompt = example["prompt"]
    inputs = ft_tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = ft_model.generate(**inputs, max_new_tokens=30)
    decoded = ft_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the generated subject line
    prediction = decoded.split("### Subject Line:")[-1].strip()

    preds.append(prediction)
    refs.append(example['subject_line'])

    # print(f"\nExample {i+1}:")
    # print(f"Predicted Subject Line: {prediction}")
    # print(f"Actual Subject Line:    {example['subject_line']}")# Create DataFrame

df = pd.DataFrame({
    "Predicted Subject Line": preds,
    "Actual Subject Line": refs
})

# Save to CSV
df.to_csv("predicted_vs_actual_subjects_for_ft_model.csv", index=False)

#Evaluate
import evaluate
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=preds, references=refs)
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

100%|██████████| 1000/1000 [39:54<00:00,  2.39s/it]


rouge1: 0.1621
rouge2: 0.0658
rougeL: 0.1493
rougeLsum: 0.1490


# **merging adapter weights with base model**

In [27]:
merged_model = ft_model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


# **Inference from the standalone merged model**

In [41]:
from tqdm import tqdm
def format_prompt(example):
    return {
        "prompt": f"### Instruction:\nGenerate a complete and non-repetitive subject line for the following email body within 20 words.\
         Keep the subject concise and relevant.\
         Exclude any details, sender/recipient information and also exclude the full email body content from the subject line,\
         Generate a brief subject line only.\
         \n\n### Email Body:\n{example['email_body']}\n\n### Subject Line:"
    }

test_dataset = test_dataset.map(format_prompt)
preds = []
refs = []

# Inference loop
for i, example in enumerate(tqdm(test_dataset)):  # change range using .select(range(k))
    prompt = example["prompt"]
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = merged_model.generate(**inputs, max_new_tokens=30)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract the generated subject line
    prediction = decoded.split("### Subject Line:")[-1].strip()

    preds.append(prediction)
    refs.append(example['subject_line'])

    # print(f"\nExample {i+1}:")
    # print(f"Predicted Subject Line: {prediction}")
    # print(f"Actual Subject Line:    {example['subject_line']}")

df = pd.DataFrame({
    "Predicted Subject Line": preds,
    "Actual Subject Line": refs
})

# Save to CSV
df.to_csv("predicted_vs_actual_subjects_for_merged_model.csv", index=False)

#Evaluate
import evaluate
rouge = evaluate.load("rouge")
results = rouge.compute(predictions=preds, references=refs)
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

100%|██████████| 1000/1000 [39:19<00:00,  2.36s/it]


rouge1: 0.1616
rouge2: 0.0640
rougeL: 0.1495
rougeLsum: 0.1492
